In [41]:
import numpy as np
import random
import math

'''g - growth 
s - switching
b - branching
r - retracting
d - dieing
n - new'''

'''
g + b + s = 1

'''

g_d_rate = 0.1
angle = 0.523599
population = ['g', 's', 'b']

def find_childless(nodes):
    childless_nodes_key = []
    for i in range(len(nodes)):
        if len(nodes[i]['children']) == 0:
            childless_nodes_key.append(i)
    return childless_nodes_key

def find_node_parent(node_id, nodes):
    for i in range(len(nodes)):
        if node_id in nodes[i]['children']:
            return nodes[i]

def check_node_near_food(node, food):
    for i in range(len(food)):
        if np.linalg.norm(node['cord'] - np.array(food[i]['cord'])) < food[i]['r']:
            return [True, food[i]['type']]
    return [False]

def delete_children_nodes(node_id, nodes):
    for i in range(len(nodes)):
        if i in nodes[i]['children']:
            nodes[i]['children'].remove(i)
    return nodes


def find_closest_food(node, food):
    max_val = float('inf')
    for i in range(len(food)):
        if np.linalg.norm(node['cord'] - np.array(food[i]['cord'])) < max_val:
            closest_food = food[i]
    return closest_food

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def create_new_node(prent_node_id, nodes, food):
    closest_food = find_closest_food(nodes[prent_node_id], food)
    axis = nodes[prent_node_id]['cord']
    v = -axis + closest_food['cord']
    v = (v/np.linalg.norm(v))*g_d_rate
    v_1 = np.dot(rotation_matrix(axis, angle), v)
    v_2 = np.dot(rotation_matrix(axis, -angle), v)
    max_key = max(list(nodes.keys()))
    nodes[max_key+1] = {
        'cord': axis + v_1,
        'children': [],
        'state': 'g'
    }
    nodes[max_key+2] = {
        'cord': axis + v_2,
        'children': [],
        'state': 'g'
    }
    nodes[prent_node_id]['children'].extend([max_key+1, max_key+2])
    return nodes


def get_csv(t, p_neg, p_pos, p_neu, b_pos, b_neg, b_neu, food):
    nodes = {}
    nodes[0] = {
        'cord': np.array([1, 2, 3]),
        'children': [],
        'state': 'g'
    }
    childless_nodes_key = find_childless(nodes)
    for j in range(t):
        for i in range(len(childless_nodes_key)):
            if nodes[i]['state'] == 'r':
                parent = find_node_parent(i, nodes)['cord']
                if np.linalg.norm(parent - nodes[i]['cord']) == g_d_rate:
                    del nodes[i]
                    nodes = delete_children_nodes(i, nodes)
                else:
                    nodes[i]['cord'] = nodes[i]['cord'] - g_d_rate
            else:
                is_near_food = check_node_near_food(nodes[i], food)
                if is_near_food[0] and is_near_food[1]:
                    weights = p_pos
                elif is_near_food[0] and (is_near_food[1] == False):
                    weights = p_neg
                else:
                    weights = p_neu
                new_state = random.choices(population, weights=weights, k=1)
                if new_state == 's':
                    nodes[i]['state'] = 'r'
                else:
                    nodes[i]['state'] = new_state
                    if new_state == 'g':
                        parent = find_node_parent(i, nodes)['cord']
                        nodes[i]['cord'] = parent + (parent - (nodes[i]['cord'] + g_d_rate))
                    else:
                        create_new_node(i, nodes, food)
    print(nodes)

In [42]:
t = 10
p_neg = [0.5, 0.45, 0.05]
p_pos = [0.5, 0.45, 0.05]
p_neu = [0.5, 0.45, 0.05]
b_pos = 1
b_neg = 1
b_neu = 1
food = [{'cord':np.array([1,1,0]), 'r':1}, {'cord':np.array([2,2,0]), 'r': 1}, {'cord':np.array([3,3,0]), 'r':1}]
get_csv(t, p_neg, p_pos, p_neu, b_pos, b_neg, b_neu, food)

{0: {'cord': array([1, 2, 3]), 'children': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'state': ['g']}, 1: {'cord': array([1.01286933, 2.05273078, 2.91601283]), 'children': [], 'state': 'g'}, 2: {'cord': array([1.07715507, 1.98844504, 2.93744141]), 'children': [], 'state': 'g'}, 3: {'cord': array([1.01286933, 2.05273078, 2.91601283]), 'children': [], 'state': 'g'}, 4: {'cord': array([1.07715507, 1.98844504, 2.93744141]), 'children': [], 'state': 'g'}, 5: {'cord': array([1.01286933, 2.05273078, 2.91601283]), 'children': [], 'state': 'g'}, 6: {'cord': array([1.07715507, 1.98844504, 2.93744141]), 'children': [], 'state': 'g'}, 7: {'cord': array([1.01286933, 2.05273078, 2.91601283]), 'children': [], 'state': 'g'}, 8: {'cord': array([1.07715507, 1.98844504, 2.93744141]), 'children': [], 'state': 'g'}, 9: {'cord': array([1.01286933, 2.05273078, 2.91601283]), 'children': [], 'state': 'g'}, 10: {'cord': array([1.07715507, 1.98844504, 2.93744141]), 'children': [], '

In [6]:
len(nodes)

1

In [11]:


v = [3, 5, 0]
axis = [4, 4, 1]
theta = 1.2 

print(np.dot(rotation_matrix(axis, theta), v)) 
# [ 2.74911638  4.77180932  1.91629719]


[2.74911638 4.77180932 1.91629719]


In [36]:
nodes

{1: {'cord': [1, 2, 3], 'children': [], 'state': 'n'}}

In [40]:
max(list(nodes.keys()))

1